In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from env import host, user, password
import acquire
import wrangle

In [16]:
df = acquire.get_zillow_data()

In [17]:
df = wrangle.handle_nulls(df)

In [18]:
df.isnull().sum()

parcelid                        0
bathroomcnt                     0
bedroomcnt                      0
buildingqualitytypeid           0
calculatedfinishedsquarefeet    0
finishedsquarefeet12            0
fips                            0
fullbathcnt                     0
latitude                        0
longitude                       0
lotsizesquarefeet               0
propertycountylandusecode       0
rawcensustractandblock          0
regionidcity                    0
regionidcounty                  0
regionidzip                     0
roomcnt                         0
unitcnt                         0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
assessmentyear                  0
landtaxvaluedollarcnt           0
taxamount                       0
logerror                        0
heatingorsystemdesc             0
propertylandusedesc             0
dtype: int64

In [6]:
df = wrangle.add_counties(df)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75206 entries, 0 to 76724
Data columns (total 29 columns):
parcelid                        75206 non-null int64
bathroomcnt                     75206 non-null float64
bedroomcnt                      75206 non-null float64
buildingqualitytypeid           75206 non-null float64
calculatedfinishedsquarefeet    75206 non-null float64
finishedsquarefeet12            75206 non-null float64
fullbathcnt                     75206 non-null float64
latitude                        75206 non-null float64
longitude                       75206 non-null float64
lotsizesquarefeet               75206 non-null float64
propertycountylandusecode       75206 non-null object
rawcensustractandblock          75206 non-null float64
regionidcity                    75206 non-null float64
regionidcounty                  75206 non-null float64
regionidzip                     75206 non-null float64
roomcnt                         75206 non-null float64
unitcnt       

In [9]:
df.to_csv('zillow_df.csv')

In [12]:
df = pd.read_csv('zillow_df.csv')  

In [19]:
df

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,latitude,longitude,...,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,logerror,heatingorsystemdesc,propertylandusedesc
0,14297519,3.5,4.0,6.0,3100.0,3100.0,6059.0,3.0,33634931.0,-117869207.0,...,1.0,1998.0,485713.0,1023282.0,2016.0,537569.0,11013.72,0.025595,None,Single Family Residential
1,17052889,1.0,2.0,6.0,1465.0,1465.0,6111.0,1.0,34449266.0,-119281531.0,...,1.0,1967.0,88000.0,464000.0,2016.0,376000.0,5672.48,0.055619,None,Single Family Residential
2,14186244,2.0,3.0,6.0,1243.0,1243.0,6059.0,2.0,33886168.0,-117823170.0,...,1.0,1962.0,85289.0,564778.0,2016.0,479489.0,6488.30,0.005383,None,Single Family Residential
3,12177905,3.0,4.0,8.0,2376.0,2376.0,6037.0,3.0,34245180.0,-118240722.0,...,1.0,1970.0,108918.0,145143.0,2016.0,36225.0,1777.51,-0.103410,Central,Single Family Residential
4,10887214,3.0,3.0,8.0,1312.0,1312.0,6037.0,3.0,34185120.0,-118414640.0,...,1.0,1964.0,73681.0,119407.0,2016.0,45726.0,1533.89,0.006940,Central,Condominium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76720,11000655,2.0,2.0,6.0,1286.0,1286.0,6037.0,2.0,34245368.0,-118282383.0,...,1.0,1940.0,70917.0,354621.0,2016.0,283704.0,4478.43,0.020615,Central,Single Family Residential
76721,17239384,2.0,4.0,6.0,1612.0,1612.0,6111.0,2.0,34300140.0,-118706327.0,...,1.0,1964.0,50683.0,67205.0,2016.0,16522.0,1107.48,0.013209,None,Single Family Residential
76722,12773139,1.0,3.0,4.0,1032.0,1032.0,6037.0,1.0,34040895.0,-118038169.0,...,1.0,1954.0,32797.0,49546.0,2016.0,16749.0,876.43,0.037129,Central,Single Family Residential
76723,12826780,2.0,3.0,6.0,1762.0,1762.0,6037.0,2.0,33937685.0,-117996709.0,...,1.0,1955.0,140000.0,522000.0,2016.0,382000.0,6317.15,0.007204,Central,Single Family Residential
